# Machine Learning Workflow Iteration 2

Content for my website: <a href="https://sdiehl28.netlify.com/" target="_blank">Software Nirvana</a>

How to use Pandas methods will not be discussed here.  Here are links to notebooks I created to assist in understanding Pandas methods.  The material on my website and notebooks is intended to suppliment a course in machine learning, rather than be a course in machine learning.  
* [github repo](https://github.com/sdiehl28/tutorial-jupyter-notebooks)  
* [Pandas Series](http://nbviewer.jupyter.org/github/sdiehl28/tutorial-jupyter-notebooks/blob/master/pandas/Series.ipynb)  
* [Pandas Axes Specification](http://nbviewer.jupyter.org/github/sdiehl28/tutorial-jupyter-notebooks/blob/master/pandas/AxisSpecification.ipynb)  
* [Pandas DataFrame](http://nbviewer.jupyter.org/github/sdiehl28/tutorial-jupyter-notebooks/blob/master/pandas/Dataframe.ipynb)  

SD TODO: Ajust link to 1st iteration of workflow on blog  
SD TODO: Define and/or provide link to "test set data leakage" and perhaps overfitting.

### Where We Are
In the first iteration, we created a simple model and showed that the accuracy was better than the null model.  The null model being the model that predicts the predominant class in all cases.

### What's Next
Refine any or all of the model building steps and measure the accuracy of the new model.

Start by copying the previous notebook and renaming it.  For each reminder to ourself on what to try next from the previous iteration, try it.  In addition, more Exploratory Data Analysis would likely be helpful.

<a name="outline"></a>
### Outline
1. [Previous Iteration](#previous)
2. [Exploratory Data Analysis](#eda)
3. [Preprocessing](#preprocess)
4. [Model Building](#model)
5. [Model Evaluation](#eval)
6. [Summary](#summary)

### Common Imports and Notebook Setup

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
%matplotlib inline
sns.set() # enable seaborn style

### Check Software Versions

In [5]:
import sys
print('python:     ', sys.version)
print('numpy:      ', np.__version__)
print('pandas:     ', pd.__version__)
import matplotlib
print('matplotlib: ', matplotlib.__version__)
print('seaborn:    ', sns.__version__)
print('sklearn:    ', sk.__version__)

python:      3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0]
numpy:       1.14.1
pandas:      0.22.0
matplotlib:  2.1.2
seaborn:     0.8.1
sklearn:     0.19.1


### Previous Iteration

In [6]:
# read in all the labeled data
all_data = pd.read_csv('../data/train.csv')

In [7]:
# break up the dataframe into X and y
# X is a 2 dimensional "spreadsheet" of values used for prediction
# y is a 1 dimensional vector of target (aka response) values
X = all_data.drop('Survived', axis=1)
y = all_data['Survived']
print('X Shape: ', X.shape)
print('y Shape: ', y.shape)

X Shape:  (891, 11)
y Shape:  (891,)


In [8]:
# create the train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=111)

******
### This Section Deals with a Subtle Train/Test Split Bug
The train/test split should create *copies* of a dataframe rather than a view into a dataframe.  However in the version I am using, a view is returned for X.

In [9]:
print(X_train.is_copy)
print(X_test.is_copy)
print(y_train.is_copy)
print(y_test.is_copy)

<weakref at 0x7f53cb2067c8; to 'DataFrame' at 0x7f540481f8d0>
<weakref at 0x7f53cb2067c8; to 'DataFrame' at 0x7f540481f8d0>
None
None


In the above we see a weakref, this means we have a view.  That will cause problems with chained assignment later.  See: [Setting With Copy Warning](https://www.dataquest.io/blog/settingwithcopywarning/)

In [12]:
# Shouldn't have to do this, but it ensures we have independent copies
X_train = X_train.copy()
X_test = X_test.copy()
y_train = y_train.copy()
y_test = y_test.copy()

In [11]:
print(X_train.is_copy)
print(X_test.is_copy)
print(y_train.is_copy)
print(y_test.is_copy)

None
None
None
None


This is correct.  If DataFrame.is_copy is None, then we are not using a view into another DataFrame.
******

<a name="eda"></a>
### Exploratory Data Analysis
[Back to Outline](#outline)

One of the first things to check for is **null values**.

In [6]:
# Find the percentage of missing values per column
nrows, ncols = X_train.shape
X_train.isnull().sum() / nrows

PassengerId    0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.187801
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.776886
Embarked       0.003210
dtype: float64

### Null Value Analysis
The following is a reasonable judgement call as to how to proceed based on the observed percentages of null values.
1. The Age attribute has some missing values => impute missing values
2. Most of the Cabin attribute is missing => remove it
3. Very few Emarked records are missing => remove records with missing Emarked value

Age imputation is likely to be helpful, however in this first iteration the goal is to quickly create a model for baseline purposes.

Write a note to ourself and others about what to try next.

**TODO: Try Age Imputation**

### Impute Age Value
For instructional purposes, to emphasize not looking at the test data, this this will be performed manually rather than using a Scikit Learn Imputer.

In [13]:
# Get a boolean Series where Age is null
train_age_null = X_train['Age'].isnull()

In [14]:
# Verify we did this correctly
X_train.loc[train_age_null, 'Age'].head()

573   NaN
697   NaN
601   NaN
709   NaN
783   NaN
Name: Age, dtype: float64

In [15]:
# Train dataset will be used to train model
# Set the null Age values, in train, to the mean Age value, in train
X_train.loc[train_age_null, 'Age'] = X_train['Age'].mean()

In [17]:
# Observe the mean value
X_train['Age'].mean()

29.787885375494064

In [18]:
# double check that the values that were null are now the mean
X_train.loc[train_age_null, 'Age'].head()

573    29.787885
697    29.787885
601    29.787885
709    29.787885
783    29.787885
Name: Age, dtype: float64

### Replace Test Set Null Age Values with Mean from *Train* Set
This step is key to understanding how to avoid "test set data leakage".  If we look at the data in the test set, in any way, it no longer acts as a test set.

We must replace null values in the test set with the mean from the *train* set without looking at any of the values in the *test* set.

In [19]:
# Test dataset will be used to evaluate the model's accuracy
# Set the null Age values, in test, to the mean Age value, in train
test_age_null = X_test['Age'].isnull()
X_test.loc[test_age_null, 'Age'] = X_train['Age'].mean()

In [20]:
# double check that the values that were null are now the mean
X_test.loc[test_age_null, 'Age'].head()

584    29.787885
411    29.787885
826    29.787885
384    29.787885
692    29.787885
Name: Age, dtype: float64

### Summary of Imputation
It is critical to understand that we replaced null values in the test set with values computed from the train set.  We never looked at the data in test set

Although we did not use Scikit Learn's imputer, it does exactly this.

### Examine Datatypes
Often this involves converting text or integers to categorical variables.

Based on a review of the data dictionary at [titanic](https://www.kaggle.com/c/titanic/data), and an examination of the values of each column, the following variables need to be converted to categorical:

** TODO: convert the following variables to categorical **
* Pclass
* Sex
* Embarked


In [24]:
# For 2nd Iteration only, ignore all text and categorical variables
X_train = X_train.drop('Pclass', axis=1)
X_test = X_test.drop('Pclass', axis=1)
X_train = X_train.drop('Name', axis=1)
X_test = X_test.drop('Name', axis=1)
X_train = X_train.drop('Sex', axis=1)
X_test = X_test.drop('Sex', axis=1)
X_train = X_train.drop('Ticket', axis=1)
X_test = X_test.drop('Ticket', axis=1)
X_train = X_train.drop('Embarked', axis=1)
X_test = X_test.drop('Embarked', axis=1)
X_train = X_train.drop('Cabin', axis=1)
X_test = X_test.drop('Cabin', axis=1)

A natural question to ask is, wouldn't it have been easier to drop these columns prior
to creating the train/test split so we wouldn't have to apply the same operation (drop column) to one?  The answer is "yes", but the proper way to do this, while ensuring no "test data leakage", is by way of pipelines and that will be discussed in a subsequent notebook.

In [22]:
# Examine the datatypes of each remaining column
X_train.dtypes

PassengerId      int64
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Cabin           object
dtype: object

<a name="preprocess"></a>
### Preprocessing
[Back to Outline](#outline)

Preprocessing was done "inline" with the Exploratory Data Analysis above.

<a name="model"></a>
### Model Building
[Back to Outline](#outline)

Perhaps the simplest model to try for classification of two classes (Survived, Not-Survived) is Logistic Regression.

Special techniques are required if one class is much more rare than another.  Let's check for that.

In [11]:
y.value_counts()

0    549
1    342
Name: Survived, dtype: int64

That's close enough to "even" for this first iteration.  Logistic Regression may work fine.

In [25]:
# Build Model
from sklearn.linear_model import LogisticRegression
base_model = LogisticRegression()
base_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

<a name="eval"></a>
### Model Evaluation
[Back to Outline](#outline)

The simplest measure of "model value" (to the business or end user) is the percent of correct predictions.  Assuming that the cost of a false positive is equal to the cost of a false negative, "accuracy" is a good measure of model value.

In [26]:
predictions = base_model.predict(X_test)

In [27]:
from sklearn.metrics import confusion_matrix
confusion_matrix(predictions, y_test)

array([[155,  69],
       [ 16,  28]])

In [28]:
# Compute Accuracy
base_model_accuracy = (155 + 28) / (155+69+16+28)
print(base_model_accuracy)

0.6828358208955224


In [16]:
# Compare with Simplest Possible Model Sometimes called the Null Model
# Null Model Predicts predominant class every time
y_test.value_counts()

0    171
1     97
Name: Survived, dtype: int64

In [17]:
# Null Model Accuracy
null_accuracy = 171 / (171 + 97)
print(null_accuracy)

0.6380597014925373


<a name="summary"></a>
### Conclusion
[Back to Outline](#outline)

The simple Logistic Regression model had a prediction accuracy of about 68%.  The null model which just predicts the most common class in all cases was accurate about 64% of the time.

In this first iteration:
* we quickly created a model
* noted a few things to try to improve the model
* established a baseline accuracy of 68%
* showed that this accuracy is better than the null model accuracy of 64%